In [1]:
import requests
from tqdm import tqdm
import pandas as pd
import numpy as np
import time
import my_utils as mu
import math
import random
import imp
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.graph_objs as go
import plotly.express as px
imp.reload(mu)
tqdm.pandas()

C:\Users\user\AppData\Local\Temp\ipykernel_8148\3501253536.py:9: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [ ]:
raw_data = mu.get_rawdata('DIAMOND')

In [ ]:
def get_pid(lane, team_num):
    if (lane == 'TOP' and team_num == 100):
        return 1
    elif (lane == 'MID' and team_num == 100):
        return 3
    elif (lane == 'BOT' and team_num == 100):
        return 4
    elif (lane == 'TOP' and team_num == 200):
        return 6
    elif (lane == 'MID' and team_num == 200):
        return 8
    elif (lane == 'BOT' and team_num == 200):
        return 9
    else:
        return None

In [ ]:
def assi_calc(x):
    try:
        return ','.join(list(map(lambda y : str(y),x['assistingParticipantIds'])))
    except:
        return ''

In [ ]:
def lane_processing(tower_lane, tower_team, lane, team):
    if (tower_lane == lane) and (tower_team == team):
        return 1
    else:
        return 0

In [ ]:
all_lst = []
try: 
    for i, game in enumerate(raw_data['matches']):
        for ids in range(len(game['info']['participants'])):
            game_id = raw_data.iloc[i]['match_id']
            game_duration = game['info']['gameDuration']
            game_version = game['info']['gameVersion']
            participantId = game['info']['participants'][ids]['participantId']
            teamPosition = game['info']['participants'][ids]['teamPosition']
            teamId = game['info']['participants'][ids]['teamId']
            championName = game['info']['participants'][ids]['championName']
            win = game['info']['participants'][ids]['win']
            kills = game['info']['participants'][ids]['kills']
            deaths = game['info']['participants'][ids]['deaths']
            assists = game['info']['participants'][ids]['assists']
            damageDealt = game['info']['participants'][ids]['totalDamageDealtToChampions']
            damageTaken = game['info']['participants'][ids]['totalDamageTaken']

            ban_lst= [banid['championId'] for teamid in game['info']['teams'] for banid in teamid['bans']]

            bans = ban_lst[ids]

            k_log, v_log, a_log,tower_log = [], [], [], []
            for frame in raw_data.iloc[i]['timeline']['info']['frames']:
                for event in frame['events']:
                    try:
                        event_type = event['type']
                        if (event_type == 'BUILDING_KILL' and event['buildingType'] == 'TOWER_BUILDING'):
                            tower_log.append(event)
                        if event_type == 'CHAMPION_KILL':
                            k_log.append(str(event['killerId']))
                            v_log.append(str(event['victimId']))
                            a_log.append(assi_calc(event))
                    except Exception as e:
                        print(e)
                        continue
            
            try:  
                ft_lane = tower_log[0]['laneType'].split('_')[0]
                ft_team_num = tower_log[0]['teamId']
                if (get_pid(ft_lane,ft_team_num) == participantId):
                    firstDT = 1
                else: firstDT = 0
            except Exception as e: 
                firstDT = 0
                continue

            laneTower = 0
            timestamp_lst =[]
            for t_log in tower_log:
                lane = t_log['laneType'].split('_')[0]
                team_num = t_log['teamId']
                if get_pid(lane, team_num) == participantId:
                    laneTower+=1
                    timestamp_lst.append(str(t_log['timestamp']))
            timestamp ='|'.join(timestamp_lst)
            
            try:
                g_15 = raw_data.iloc[i].timeline['info']['frames'][14]['participantFrames'][str(ids+1)]['totalGold']
            except:
                g_15 = 0

            k_log, v_log, a_log = '|'.join(k_log), '|'.join(v_log), '|'.join(a_log)
            all_lst.append([game_id, game_duration, game_version, participantId, championName, teamPosition, teamId,win,kills,deaths,assists,damageDealt,damageTaken,firstDT, laneTower, timestamp, bans, k_log, v_log, a_log, g_15])
except Exception as e: 
    print(e)
    pass
df = pd.DataFrame(all_lst,columns=['matchId','gameDuration','gameVersion', 'participantId','championName','teamPosition', 'teamId','win','kills','deaths', 'assists',
         'damageDealt','damageTaken','firstDT','laneTower', 'laneTowerTime','bans','killerId','victimId','assistId','G_15'])

In [6]:
conn = mu.connect_mysql('icia')
df = pd.DataFrame(mu.mysql_execute_dict('select * from lol_datas',conn))
conn.close()
df = df[(df.gameDuration > 900) & (df.gameDuration <10000)]

In [231]:
sample = df[['gameId','championName','win','teamPosition','g_15','teamId','kills','deaths','assists','totalDamageDealtToChampions']]

In [232]:
# 챔피언이 해당 라인에서 승리한 횟수
#sample['champ_in_lane_win_cnt'] = sample.groupby(['championName', 'teamPosition'])[['win']].transform('sum')

In [233]:
# 챔피언이 해당 라인에서 등장한 횟수
#sample['champ_in_lane_cnt'] = sample.groupby(['championName', 'teamPosition'])[['win']].transform('count')

In [234]:
# 라인 횟수
#sample[['win']].count()/5

In [235]:
# 각 팀 킬 수 합쳐서 team_kills 컬럼에 추가
sample['team_kills'] = sample.groupby(['gameId', 'teamId'])['kills'].transform('sum')

C:\Users\user\AppData\Local\Temp\ipykernel_8148\2933603564.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['team_kills'] = sample.groupby(['gameId', 'teamId'])['kills'].transform('sum')


In [236]:
#win을 boolean에서 int로 변경
sample['win'] =sample.apply(lambda x:  1 if x.win =='True' else 0 , axis = 1)

C:\Users\user\AppData\Local\Temp\ipykernel_8148\4082881006.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample['win'] =sample.apply(lambda x:  1 if x.win =='True' else 0 , axis = 1)


In [237]:
#팀별로 테이블 분리
blue = sample[sample['teamId']==100]
red =sample[sample['teamId']==200]

In [238]:
#팀별 컬럼 리네임
blue_tmp = blue[['gameId','championName','teamPosition','g_15','kills','deaths','assists','team_kills']].rename(columns={'championName':'enemy_champ','g_15':'enemy_g_15','kills':'enemy_kills', 'deaths':'enemy_deaths', 'assists':'enemy_assists', 'team_kills':'enemy_team_kills'})
red_tmp = red[['gameId','championName','teamPosition','g_15','kills','deaths','assists','team_kills']].rename(columns={'championName':'enemy_champ','g_15':'enemy_g_15','kills':'enemy_kills', 'deaths':'enemy_deaths', 'assists':'enemy_assists', 'team_kills':'enemy_team_kills'})

In [239]:
#분리한 팀별 테이블 merge해서 비교
blue_merge = pd.merge(blue,red_tmp, on=['gameId','teamPosition'])
red_merge = pd.merge(red,blue_tmp, on=['gameId','teamPosition'])

In [240]:
# append를 통해서 다시 하나의 테이블로
result = blue_merge.append(red_merge)

C:\Users\user\AppData\Local\Temp\ipykernel_8148\1241871353.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = blue_merge.append(red_merge)


In [241]:
# 인덱스 재설정
result = result.reset_index(drop=True)

In [242]:
# 게임아이디랑 팀 아이디 별로 정렬
result = result.sort_values(by=['gameId','teamId'])

In [243]:
win_df = result.copy()

In [23]:
# 라인승리 컬럼 값을 15분대의 골드획득량으로 승패 결정
# win_df['lane_win'] = win_df.apply(lambda x: 1 if x.g_15>x.enemy_g_15 else 0 , axis=1)

In [244]:
# 팀포지션컬럼을 라인으로 리네임
win_df.rename(columns={'teamPosition': 'lane'},inplace=True)

In [245]:
# 챔피언 이름 , 라인, 적 챔피언 순으로 그룹화 해서 라인승리 누적횟수와 게임승리 누적횟수를 보여줌
cnt_df = win_df.groupby(['championName','lane','enemy_champ'])[['win','g_15','kills','deaths','assists','totalDamageDealtToChampions','team_kills','enemy_kills','enemy_deaths','enemy_assists','enemy_team_kills']].sum().rename(columns={'win':'win_cnt'})

In [247]:
# 전체 게임수를 카운트해서 game_cnt라는 컬럼의 값으로 넣음
game_df = win_df.groupby(['championName','lane','enemy_champ'])[['win']].count().rename(columns={'win':'game_cnt'})

In [248]:
#merge를 통해 전체 게임수와 누적 승리를 보여줌
result_df = pd.merge(cnt_df,game_df,on=['championName','lane','enemy_champ'])

In [249]:
# 게임수가 많은 순으로 정렬
r_df = result_df.sort_values('game_cnt',ascending=False)

In [250]:
# 승률
r_df['win_rate']=round((r_df['win_cnt']/r_df['game_cnt'])*100,2)

In [251]:
#kda
r_df['kda']=round((r_df['kills']+r_df['assists'])/r_df['deaths'],2)

In [252]:
#라인킬 확률
r_df['lane_kills'] = round((r_df['kills']/r_df['deaths'])/((r_df['kills']/r_df['deaths'])+(r_df['enemy_kills']/r_df['enemy_deaths']))*100,2)

In [253]:
#킬관여율
r_df['kill_participation'] = round(( ((r_df['kills']+r_df['assists'])/r_df['team_kills']) /(((r_df['kills']+r_df['assists'])/r_df['team_kills']) + ((r_df['enemy_kills']+r_df['enemy_assists'])/r_df['team_kills']))  )*100,2)

In [254]:
#챔피언에게 가한 피해량
r_df['deal_to_champ'] = round(r_df['totalDamageDealtToChampions']/r_df['game_cnt'])

In [255]:
#15분대의 골드 획득량
r_df['avg_g_15'] = round(r_df['g_15']/r_df['game_cnt'])

In [ ]:
#라인첫번째 타워를 파괴한 시간

In [256]:
r_df_o = r_df[['lane_kills','kda','kill_participation','deal_to_champ','win_rate','win_cnt','game_cnt']]

In [162]:
# multi-level index로 설정된 경우 
r_df_o.loc[r_df_o.index.get_level_values('championName') == 'Yorick']

lane_kills   kda  kill_participation  \
championName lane enemy_champ                                         
Yorick       TOP  Illaoi            43.69  1.27               47.01   
                  Jax               48.24  1.52               50.00   
                  Aatrox            58.23  1.85               50.00   
                  Mordekaiser       36.23  1.09               41.00   
                  Renekton          23.59  1.25               38.53   
...                                   ...   ...                 ...   
                  Velkoz             0.00  0.40                8.00   
                  Veigar             7.69  0.67               18.18   
                  Urgot             10.47  0.78               19.44   
                  Talon             82.09  5.00               57.69   
                  Shaco             57.14  3.00               50.00   

                               deal_to_champ  win_rate  game_cnt  
championName lane enemy_champ                                     
Yorick       TOP  Illaoi             23084.0     42.11        19  
                  Jax                19520.0     63.16        19  
                  Aatrox             18625.0     57.89        19  
                  Mordekaiser        22558.0     53.33        15  
                  Renekton           19126.0     35.71        14  
...                                      ...       ...       ...  
                  Velkoz             17173.0      0.00         1  
                  Veigar              6841.0      0.00         1  
                  Urgot              31601.0      0.00         1  
                  Talon              20623.0    100.00         1  
                  Shaco              13649.0      0.00         1  

[110 rows x 6 columns]

In [257]:
r_df_o

,,,lane_kills,kda,kill_participation,deal_to_champ,win_rate,win_cnt,game_cnt
championName,lane,enemy_champ,,,,,,,
Ezreal,BOTTOM,Kaisa,42.41,2.44,47.90,23835.0,46.97,209,445
Kaisa,BOTTOM,Ezreal,57.59,2.69,52.10,21514.0,53.03,236,445
Caitlyn,BOTTOM,Ezreal,49.78,2.42,50.37,19463.0,55.34,197,356
Ezreal,BOTTOM,Caitlyn,50.22,2.46,49.63,23023.0,44.66,159,356
Jhin,BOTTOM,Ezreal,58.18,3.18,54.45,19443.0,54.88,180,328
...,...,...,...,...,...,...,...,...,...
Kaisa,BOTTOM,Nautilus,90.91,7.33,68.75,21934.0,100.00,1,1
Sett,TOP,Khazix,94.74,6.00,63.16,18098.0,100.00,1,1
Kaisa,BOTTOM,Nunu,96.97,6.33,73.08,23116.0,100.00,1,1
